<a href="https://colab.research.google.com/github/lefkothea1/NLP_2/blob/main/Copy_of_NLP_ass2_ex3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [13]:
#to select GPU: Edit → Notebook Settings

#loading data from github repo- WILL USE GOOGLE DRIVE
# import pandas as pd
# url_diagn_test = 'https://raw.githubusercontent.com/lefkothea1/NLP_2/main/data/Assignment%202_IntrotoNLP2022_data/olid-subset-diagnostic-tests.csv'
# url_test = 'https://raw.githubusercontent.com/lefkothea1/NLP_2/main/data/Assignment%202_IntrotoNLP2022_data/olid-test.csv'
# url_train ='https://raw.githubusercontent.com/lefkothea1/NLP_2/main/data/Assignment%202_IntrotoNLP2022_data/olid-train.csv'
# diagnostic_test = pd.read_csv(url_test)
# train = pd.read_csv(url_train)
# test = pd.read_csv(url_test)

#setting up google drive and loading all data
from google.colab import drive

drive.mount('/content/drive')
train = pd.read_csv("/content/drive/MyDrive/NLP2/data/Assignment 2_IntrotoNLP2022_data/olid-train.csv")
test = pd.read_csv("/content/drive/MyDrive/NLP2/data/Assignment 2_IntrotoNLP2022_data/olid-test.csv")
diagn_test = pd.read_csv("/content/drive/MyDrive/NLP2/data/Assignment 2_IntrotoNLP2022_data/olid-subset-diagnostic-tests.csv")

print(train.shape)
print(train.head())


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
(13240, 3)
      id                                               text  labels
0  86426  @USER She should ask a few native Americans wh...       1
1  90194  @USER @USER Go home you’re drunk!!! @USER #MAG...       1
2  16820  Amazon is investigating Chinese employees who ...       0
3  62688  @USER Someone should'veTaken" this piece of sh...       1
4  43605  @USER @USER Obama wanted liberals &amp; illega...       0


In [23]:
#setting up- from https://simpletransformers.ai/docs/binary-classification/
import logging

logging.basicConfig(level=logging.INFO)
transformers_logger = logging.getLogger("transformers")
transformers_logger.setLevel(logging.WARNING)

In [3]:
#should i install it every time?-yes
!pip install simpletransformers 

     |████████████████████████████████| 249 kB 5.3 MB/s 
     |████████████████████████████████| 10.1 MB 42.7 MB/s 
     |████████████████████████████████| 43 kB 2.1 MB/s 
     |████████████████████████████████| 1.2 MB 35.9 MB/s 
     |████████████████████████████████| 6.6 MB 36.4 MB/s 
     |████████████████████████████████| 325 kB 48.4 MB/s 
     |████████████████████████████████| 4.0 MB 19.3 MB/s 
     |████████████████████████████████| 1.8 MB 35.5 MB/s 
     |████████████████████████████████| 596 kB 34.6 MB/s 
     |████████████████████████████████| 77 kB 6.1 MB/s 
     |████████████████████████████████| 895 kB 41.6 MB/s 
     |████████████████████████████████| 181 kB 50.6 MB/s 
     |████████████████████████████████| 144 kB 20.9 MB/s 
     |████████████████████████████████| 63 kB 1.5 MB/s 
     |████████████████████████████████| 136 kB 32.0 MB/s 
     |████████████████████████████████| 1.1 MB 40.7 MB/s 
     |████████████████████████████████| 212 kB 45.9 MB/s 
     |██████████████

In [10]:
from simpletransformers.classification import ClassificationModel, ClassificationArgs

In [21]:
# Optional model configuration-using best teams args -from paper- SHOULD I USE GRID THINGY TO CHECK?DIDNT FOR NOW
model_args = ClassificationArgs()
model_args.num_train_epochs=2
# model_args.max_sentence_length=64-not sure about this one
model_args.output_dir = '/content/drive/MyDrive/NLP2/trained_models'

In [24]:
#loading neccessary pretrained model
bert_mod = ClassificationModel('bert', 'bert-base-cased', args = model_args)

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at b

In [25]:
#training/ finetuning on OUR training data-takes time! will be done with kwargs at the end of day !!!!!!!!!!!!!!!!1!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!1
bert_mod.train_model(train)

INFO:simpletransformers.classification.classification_utils: Converting to features started. Cache is not used.


  0%|          | 0/13240 [00:00<?, ?it/s]

INFO:simpletransformers.classification.classification_utils: Saving features into cached file cache_dir/cached_train_bert_128_2_2
/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,


Epoch:   0%|          | 0/2 [00:00<?, ?it/s]

Running Epoch 0 of 2:   0%|          | 0/1655 [00:00<?, ?it/s]

KeyboardInterrupt: ignored

In [16]:
from sklearn.metrics import classification_report
#TO LOAD MODEL FROM DRIVE-NOT SAVED THERE YET!
#bert_mod = ClassificationModel("bert", "/content/drive/MyDrive/NLP2/trained_models/best_model")
# Evaluate the model
result, model_outputs, wrong_predictions = model.eval_model(test)

  0%|          | 0/860 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/108 [00:00<?, ?it/s]

In [20]:
result
model_outputs
#wrong_predictions

array([[ 0.29394531,  0.04879761],
       [ 0.43334961, -0.15319824],
       [ 1.04492188, -0.98925781],
       ...,
       [-0.55322266,  0.90380859],
       [ 0.31811523, -0.00432968],
       [ 1.1875    , -1.27832031]])

OSError: ignored